In [1]:
from io import open
import glob
import os
import unicodedata
import string
import torch

all_letters = string.ascii_letters + " .,;'-"
n_letters = len(all_letters) + 1 # Plus EOS marker

def findFiles(path): return glob.glob(path)

# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

# Read a file and split into lines
def readLines(filename):
    with open(filename, encoding='utf-8') as some_file:
        return [unicodeToAscii(line.strip()) for line in some_file]

# Build the category_lines dictionary, a list of lines per category
category_lines = {}
all_categories = []
for filename in findFiles('RNN_data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines   #******

n_categories = len(all_categories)

if n_categories == 0:
    raise RuntimeError('Data not found. Make sure that you downloaded data '
        'from https://download.pytorch.org/tutorial/data.zip and extract it to '
        'the current directory.')

print('# categories:', n_categories, all_categories)
print(unicodeToAscii("O'Néàl"))

# categories: 18 ['Italian', 'Dutch', 'German', 'Japanese', 'Greek', 'Scottish', 'Czech', 'Chinese', 'Irish', 'French', 'English', 'Korean', 'Polish', 'Spanish', 'Vietnamese', 'Portuguese', 'Russian', 'Arabic']
O'Neal


In [2]:
#%%
import torch
from torch.utils.data import Dataset, DataLoader

class DictDataset(Dataset):
    def __init__(self, data_dict):
        """
        Args:
            data_dict (dict): A dictionary where keys are feature names and values are lists/tensors of data.
        """
        self.data_dict = data_dict
        self.keys = list(data_dict.keys())
        self.length = len(next(iter(data_dict.values())))  # Get length from the first key

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        # Return a dictionary where each key has the corresponding indexed value
        return {key: torch.tensor(self.data_dict[key][idx]) for key in self.keys}

# Create Dataset
# dataset = DictDataset(category_lines)

# # Create DataLoader
# dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# # Iterate Over DataLoader
# for batch in dataloader:
#     print(batch)  # Each batch is a dictionary with tensors
    

In [3]:
#%%  RNN
import torch.nn as nn
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
torch.set_default_device(device)


class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size

        self.i2h = nn.Linear(n_categories + input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(n_categories + input_size + hidden_size, output_size)
        self.o2o = nn.Linear(hidden_size + output_size, output_size)
        self.dropout = nn.Dropout(0.1)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, category, input, hidden):
        input_combined = torch.cat((category, input, hidden), 1)
        hidden = self.i2h(input_combined)
        output = self.i2o(input_combined)
        output_combined = torch.cat((hidden, output), 1)
        output = self.o2o(output_combined)
        output = self.dropout(output)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [4]:
#%%  generate a random example
import random

# Random item from a list
def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

# Get a random category and random line from that category
def randomTrainingPair():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    return category, line

# One-hot vector for category
def categoryTensor(category):
    li = all_categories.index(category)
    tensor = torch.zeros(1, n_categories)
    tensor[0][li] = 1
    return tensor

# One-hot matrix of first to last letters (not including EOS) for input
def inputTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li in range(len(line)):
        letter = line[li]
        tensor[li][0][all_letters.find(letter)] = 1
    return tensor

# ``LongTensor`` of second letter to end (EOS) for target
def targetTensor(line):
    letter_indexes = [all_letters.find(line[li]) for li in range(1, len(line))]
    letter_indexes.append(n_letters - 1) # EOS
    return torch.tensor(letter_indexes, dtype = torch.int64)

# Make category, input, and target tensors from a random category, line pair
def randomTrainingExample():
    category, line = randomTrainingPair()
    category_tensor = categoryTensor(category)
    input_line_tensor = inputTensor(line)
    target_line_tensor = targetTensor(line)
    
    return category_tensor, input_line_tensor, target_line_tensor
    # return category_tensor.to(device), input_line_tensor.to(device), target_line_tensor.to(device)
    
import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [5]:
#%% Train
rnn = RNN(n_letters, 128, n_letters).to(device)

criterion = nn.NLLLoss()
learning_rate = 0.0005

optimizer = torch.optim.SGD(rnn.parameters(), lr=learning_rate)

def train(category_tensor, input_line_tensor, target_line_tensor):
    target_line_tensor.unsqueeze_(-1)
    hidden = rnn.initHidden()

    rnn.zero_grad()

    loss = torch.Tensor([0]).to(device) # you can also just simply use ``loss = 0``

    for i in range(input_line_tensor.size(0)):
        output, hidden = rnn(category_tensor, input_line_tensor[i], hidden)
        l = criterion(output, target_line_tensor[i])
        loss += l

    loss.backward()

    # for p in rnn.parameters():
    #     p.data.add_(p.grad.data, alpha=-learning_rate)
    
    optimizer.step()
    optimizer.zero_grad()

    return output, loss.item() / input_line_tensor.size(0)

In [ ]:
n_iters = 100000
print_every = 5000
plot_every = 500
all_losses = []
total_loss = 0 # Reset every ``plot_every`` ``iters``

start = time.time()

for idx in range(1, n_iters + 1):
    output, loss = train(*randomTrainingExample())
    total_loss += loss

    if idx % print_every == 0:
        print('%s (%d %d%%) %.4f' % (timeSince(start), idx, idx / n_iters * 100, loss))

    if idx % plot_every == 0:
        all_losses.append(total_loss / plot_every)
        total_loss = 0

0m 26s (5000 5%) 2.8783
0m 50s (10000 10%) 2.3705
1m 14s (15000 15%) 3.2459
1m 39s (20000 20%) 2.0486
2m 3s (25000 25%) 3.2222
2m 27s (30000 30%) 2.3483
2m 51s (35000 35%) 3.0740
3m 15s (40000 40%) 2.4015
3m 39s (45000 45%) 1.6394
4m 3s (50000 50%) 1.9100
4m 28s (55000 55%) 3.1187
4m 55s (60000 60%) 2.2355


In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(all_losses)
plt.show()
#%%  save and load
# torch.save(rnn.state_dict(), "RNN.pth")

# model = RNN(n_letters, 128, n_letters).to(device)
# model.load_state_dict(torch.load("RNN.pth", weights_only=True))

In [ ]:
#%% Sampling 
max_length = 20

# Sample from a category and starting letter
def sample(category, start_letter='A'):
    with torch.no_grad():  # no need to track history in sampling
        category_tensor = categoryTensor(category)
        input = inputTensor(start_letter)
        hidden = rnn.initHidden()

        output_name = start_letter

        for i in range(max_length):
            output, hidden = rnn(category_tensor, input[0], hidden)
            topv, topi = output.topk(1)
            topi = topi[0][0]
            if topi == n_letters - 1:
                break
            else:
                letter = all_letters[topi]
                output_name += letter
            input = inputTensor(letter)

        return output_name

# Get multiple samples from one category and multiple starting letters
def samples(category, start_letters='ABC'):
    for start_letter in start_letters:
        print(sample(category, start_letter))

samples('Russian', 'RUS')

samples('German', 'GER')

samples('Spanish', 'SPA')

samples('Chinese', 'CHI')